## Vector stores and retivers

This video tutorial will familliarize you with langchain vectore store and retiver abstraction .These abstraction are designed to support retrival of data -- from (vectore )databases and othersources --for intigration with  LLm workflow .The are importent for application that fetch data to be reasoned over as part of model inferance .as in the case of retrieval-augumented genration.

we will cover:

   Documents
   
   Vector Store

   Retrivers

## Documents

Langchain Implements a Documents abstraction which is indepeded to represent a unit of text and associated metadata it has two attributes:

page_content: a string representing the content ,

Metedata: a dict containing arbitory metadata. The metadata attribute can capture information about the source of the document its relationship to other documents and other information  note that an individual Document objectsoften respresent a chunks of a larger documents.

lets genrate some sample documents:

In [6]:
from langchain_core.documents import Document
documents=[
 Document(
    page_content="Dogs are great companions,known for there loyalty and freandliness",
    metadata={"source": "mammal-pet-doc"},
),
Document(
    page_content="Cats are independent pets that often enjoy there own space",
    metadata={"source": "mammal-pet-doc"},
),
 Document(
    page_content="Gold fish are popular pets for beginners,requireing relatively simple care",
    metadata={"source": "fish-pet-doc"},
),
Document(
    page_content="parrots are intelligent birds capable of mimicking human speech",
    metadata={"source": "bird-pet-doc"},
),
Document(
    page_content="Rabbits are social animals that need plenty of space to hop arround",
    metadata={"source": "mammal-pet-doc"},
),


]



In [7]:
documents

[Document(metadata={'source': 'mammal-pet-doc'}, page_content='Dogs are great companions,known for there loyalty and freandliness'),
 Document(metadata={'source': 'mammal-pet-doc'}, page_content='Cats are independent pets that often enjoy there own space'),
 Document(metadata={'source': 'fish-pet-doc'}, page_content='Gold fish are popular pets for beginners,requireing relatively simple care'),
 Document(metadata={'source': 'bird-pet-doc'}, page_content='parrots are intelligent birds capable of mimicking human speech'),
 Document(metadata={'source': 'mammal-pet-doc'}, page_content='Rabbits are social animals that need plenty of space to hop arround')]

In [21]:
import os
from dotenv import load_dotenv
from langchain_groq import ChatGroq
load_dotenv()
groq_api_key=os.getenv("Groq_key")
os.environ["HF_token"]=os.getenv("HF_token")
llm=ChatGroq(groq_api_key=groq_api_key, model="llama-3.1-8b-instant")#"Llama3-8b-8192"
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001A50FDD7610>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001A50FDD4390>, model_name='llama-3.1-8b-instant', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [22]:
from langchain_huggingface import HuggingFaceEmbeddings

embeddings=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

In [23]:
## VectoreStore 
from langchain_chroma import Chroma
vectorestore=Chroma.from_documents(documents,embedding=embeddings)
vectorestore

In [24]:
vectorestore.similarity_search("cat")

[Document(id='f582ba1f-bc87-4a2e-938f-6a3afa6e1718', metadata={'source': 'mammal-pet-doc'}, page_content='Cats are independent pets that often enjoy there own space'),
 Document(id='0e00be5e-afa3-4920-8293-69e57f95f680', metadata={'source': 'mammal-pet-doc'}, page_content='Cats are independent pets that often enjoy there own space'),
 Document(id='d3745c1c-8a84-4f97-8270-8c8aeb1cd673', metadata={'source': 'mammal-pet-doc'}, page_content='Dogs are great companions,known for there loyalty and freandliness'),
 Document(id='ad229132-6b60-474a-9b14-cae13188f869', metadata={'source': 'mammal-pet-doc'}, page_content='Dogs are great companions,known for there loyalty and freandliness')]

In [25]:
## Async Qurey
await vectorestore.asimilarity_search("cat")

[Document(id='f582ba1f-bc87-4a2e-938f-6a3afa6e1718', metadata={'source': 'mammal-pet-doc'}, page_content='Cats are independent pets that often enjoy there own space'),
 Document(id='0e00be5e-afa3-4920-8293-69e57f95f680', metadata={'source': 'mammal-pet-doc'}, page_content='Cats are independent pets that often enjoy there own space'),
 Document(id='d3745c1c-8a84-4f97-8270-8c8aeb1cd673', metadata={'source': 'mammal-pet-doc'}, page_content='Dogs are great companions,known for there loyalty and freandliness'),
 Document(id='ad229132-6b60-474a-9b14-cae13188f869', metadata={'source': 'mammal-pet-doc'}, page_content='Dogs are great companions,known for there loyalty and freandliness')]

In [26]:
## With score of maching based

vectorestore.similarity_search_with_score("cat")

[(Document(id='f582ba1f-bc87-4a2e-938f-6a3afa6e1718', metadata={'source': 'mammal-pet-doc'}, page_content='Cats are independent pets that often enjoy there own space'),
  0.9529241323471069),
 (Document(id='0e00be5e-afa3-4920-8293-69e57f95f680', metadata={'source': 'mammal-pet-doc'}, page_content='Cats are independent pets that often enjoy there own space'),
  0.9529241323471069),
 (Document(id='d3745c1c-8a84-4f97-8270-8c8aeb1cd673', metadata={'source': 'mammal-pet-doc'}, page_content='Dogs are great companions,known for there loyalty and freandliness'),
  1.5553232431411743),
 (Document(id='ad229132-6b60-474a-9b14-cae13188f869', metadata={'source': 'mammal-pet-doc'}, page_content='Dogs are great companions,known for there loyalty and freandliness'),
  1.5553232431411743)]

## Retrivers

Langchain Vectorestore object do not subclass Runnable , and so cannot immediately be intigrated into LangChain Expression Language Chain.

LangChain Retrivers are Runnables,so they implement a standerd set of methods(e.g. synchronous and asynchronous invoke and batch operations)and are designed to be incorporated in LCEL Chains.

We can create a simple version of the ourselves ,without subclassing Retriver.if we choose what method we wish to use to retrive documents,we can create a runnable easily. below we will build one around the similarity_Search Method.

In [27]:
from typing import List
from langchain_core.runnables import RunnableLambda

retriver=RunnableLambda(vectorestore.similarity_search).bind(k=1)
retriver.batch(["cat","dog"])

[[Document(id='f582ba1f-bc87-4a2e-938f-6a3afa6e1718', metadata={'source': 'mammal-pet-doc'}, page_content='Cats are independent pets that often enjoy there own space')],
 [Document(id='d3745c1c-8a84-4f97-8270-8c8aeb1cd673', metadata={'source': 'mammal-pet-doc'}, page_content='Dogs are great companions,known for there loyalty and freandliness')]]

Vectorestore Implement an as Retriver method that will genrate a retiver specifically a VectorestoreRetriver These retriver include specific search_type and Search_Kwords attribute that identify what methode of the underlying vectore store to call and how to parameterize them.for instunce we can replicate the above with the following.

In [28]:
retriver=vectorestore.as_retriever(
                          search_type="similarity",
                          search_kwargs={"k":1}
                          )
retriver.batch(["cat","dog"])

[[Document(id='f582ba1f-bc87-4a2e-938f-6a3afa6e1718', metadata={'source': 'mammal-pet-doc'}, page_content='Cats are independent pets that often enjoy there own space')],
 [Document(id='d3745c1c-8a84-4f97-8270-8c8aeb1cd673', metadata={'source': 'mammal-pet-doc'}, page_content='Dogs are great companions,known for there loyalty and freandliness')]]

#### Impliment Retriver with chain

In [ ]:
### Retriver Agument and genrator
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

message=""" 
  Answer this question using the Provided context only.
  {question}

  Context:
  {context}

"""
prompt=ChatPromptTemplate.from_messages([
    ("human",message)
])
rag_chain={"context":retriver,"question":RunnablePassthrough()}|prompt|llm

response=rag_chain.invoke("tell me about dog")
print(response.content)

Dogs are great companions. They are known for their loyalty and friendliness.
